# Programmation orientée objet

Comment créer se propres objets en Python. 

## Vocabulaire

En python tout ce que l'on manipule est un objet : 
* Nombre, liste, dictionnaire, ...
* Tableau numpy
* Fonction, module, ....

Un objet possède des **attributs**. Exemple : 

    z = 1.1J
    z.real
    a = np.array([1, 2, 4])
    a.shape
    
Un objet possède des **méthodes**. Exemple :

    a.mean()
    a.conjugate()
    z.conjugate()
    
Les méthodes sont des fonctions attachées à l'objet. La méthode conjugate d'un tableau n'est pas la même que la méthode conjugate d'un complexe

## Comment créer un objet

### Créer un nombre complexe


In [126]:
from math import sqrt, atan2, cos, sin, pi

class Complex:
    def __init__(self, partie_reelle, partie_imaginaire=0):
        self.re = partie_reelle
        self.im = partie_imaginaire
        
    def conjugate(self):
        return Complex(self.re, -self.im)
    
    def __str__(self):
        if self.im==0:
            return f'{self.re}'
        if self.im>0:
            return f'{self.re} + {self.im}J'
        if self.im<0:
            return f'{self.re} - {-self.im}J'
        
    def __repr__(self):
        return f'Complex({self.re}, {self.im})'
    
    def _make_other_complex(self, other):
        if isinstance(other, (int, float)):
            return Complex(other, 0)
        return other
    
    def __add__(self, other):
        other = self._make_other_complex(other)
        if isinstance(other, Complex):
            return Complex(self.re + other.re, self.im + other.im)
        return NotImplemented

    def __sub__(self, other):
        other = self._make_other_complex(other)
        if isinstance(other, Complex):
            return Complex(self.re - other.re, self.im - other.im)
        return NotImplemented
    
    def __truediv__(self, other):
        other = self._make_other_complex(other)
        if isinstance(other, Complex):
            return Complex.from_polar(self.r/other.r, self.phase - other.phase)
        return NotImplemented
            

    def __eq__(self, other):
        other = self._make_other_complex(other)
        if isinstance(other, Complex):
            return other.re==self.re and other.im==self.im
        return NotImplemented
    
    def __radd__(self, other):
        return self + other
    
    @property
    def r(self):
        return sqrt(self.re**2 + self.im**2)
    
    @property
    def phase(self):
        return atan2(self.im, self.re)

    @classmethod
    def from_polar(cls, r, phase):
        return Complex(r*cos(phase), r*sin(phase))

In [127]:
z1 = Complex(1, 0.5)
z2 = Complex(2.3, 1.3)

z1.phase

0.4636476090008061

In [128]:
z1/z2

Complex(0.42263610315186245, -0.021489971346704898)

In [96]:
z = Complex(1, 0.5)

print(z)

1 + 0.5J


In [68]:
z_conj = z.conjugate()
z_conj.im
print(z_conj)

1 - 0.5J


In [69]:
print(z)
str(z)

1 + 0.5J


'1 + 0.5J'

In [71]:
isinstance(z, object)

True

In [55]:
repr(z)

'Complex(1, 0.5)'

In [20]:
s = 'Bonjour'
print(s)

Bonjour


In [21]:
s

'Bonjour'

In [26]:
import numpy as np
l = ['Bonjour, Hello', np.array([1, 2])]

print(l)

['Bonjour, Hello', array([1, 2])]


In [24]:
print(l)

[1, 2]


### Dataclass

In [13]:
import dataclasses

@dataclasses.dataclass
class Book():
    titre : str
    auteur : str
    annee : int
        
book1 = Book("A very nice book", "F. Dupont", 2014)
book1

Book(titre='A very nice book', auteur='F. Dupont', annee=2014)

### Instrumentation

Exemple : oscilloscope (c.f. TP)

Inconvénients de la méthode fonctionnelle : 

* L'utilisateur doit importer beaucoup de fonction
* Les fonctions dépendent de l'instrument, il faut donc connaitre l'instrument pour savoir quelle fonction utiliser
* Il faut gérer les arguments des fonctions.

In [33]:
from fake_scpi import FakeSCPI
inst = FakeSCPI()

def set_frequency(inst, freq=1000):
    command = f'FREQ {freq:.3f}'
    inst.write(command)

def get_frequency(inst):
    command = f'FREQ?'
    return float(inst.query(command))

def get_model_identification(inst):
    res = inst.query("*IDN?")
    comp, model, serial, _ = res.split(',') 
    dic = {'compagnie':comp,
          'model':model,
          'serial':serial}
    return dic

set_frequency(inst, 100)
get_frequency(inst)
get_model_identification(inst)

{'compagnie': 'TEKTRONIX', 'model': 'DPO3014', 'serial': 'C012048'}

In [46]:
class Instrument():
    def __init__(self, conn):
        self._conn = conn
        
    def get_model_identification(self):
        res = self._conn.query("*IDN?")
        comp, model, serial, _ = res.split(',') 
        dic = {'compagnie':comp,
              'model':model,
              'serial':serial}
        return dic
    
    @property
    def idn(self):
        return self.get_model_identification()

class GBF(Instrument):
    frequency = property(lambda self:self.get_frequency(), lambda self, val:self.set_frequency(val))
    
    def get_frequency(self):
        raise Exception()
    
class TektronixGBF(GBF):
    def set_frequency(self, freq=1000):
        command = f'FREQ {freq:.3f}'
        self._conn.write(command)

    def get_frequency(self):
        command = f'FREQ?'
        return float(self._conn.query(command))
    
        
class AgilentGBF(GBF):
    def set_frequency(self, freq=1000):
        command = f'FRE {freq:.3f}'
        self._conn.write(command)

    def get_frequency(self):
        command = f'FRE?'
        return float(self._conn.query(command))


    
gbf = TektronixGBF(FakeSCPI())

In [47]:
isinstance(gbf, GBF)

True

In [48]:
gbf.frequency = 10
gbf.frequency

10.0

## Nouveaux concepts



### Méthodes spéciales

- `__init__`
- `__repr__`, `__str__`

Lorsque c'est possible, `__repr__` doit représenté au mieux l'objet. Souvent il s'agit d'une chaine de caractère qui en étant évaluée renvoie un objet similaire. La méthode `__str__` renvoie `__repr__` par défaut. Sinon, elle doit être plus simple.

Opérateur unaire et binaire

- `__neg__`
- `__add__`, `__sub__`, `__mul__`, `__truediv__`, `__mod__`, `__pow__`
- `__radd__`, ...
- `__eq__` (==), `__ne__` (!=), `__lt__` (<), `__le__` (<=), `__gt__`, `__ge__`
- `__or__` (|), `__and__` (&), `__xor__`(^)
- `__matmul__` (@)


Emulateur de contenu

- a[key] => `a.__getitem__(key)`
- a[key] = val => `a.__setitem__(key, val)`
- del a[key] => `a.__delitem__(key)`
- len(a) => `a.__len__()`
- for elm in a => `for elm in a.__iter__()`

Emulateur de fonction 

- obj(a, b) => `obj.__call__(a, b)`

### Attributs et property

- Attributs de class vs attributs d'objets
- property

### Héritage

- isinstance permet de tester si un objet est une instance d'une classe. 

In [51]:
class Test():
    def __init__(self, val):
        self._val = val
    def __getitem__(self, val):
        if isinstance(val, slice):
            return self._val[slice.start]
        print(type(val), val)
        
t = Test()

In [65]:
t[...]

<class 'ellipsis'> Ellipsis


In [61]:
a = slice(1, 2, 4)


In [64]:
a.step

4

In [76]:
class Range():
    def __init__(self, start, stop, step):
        self.start = start
        self.step = step
        self.stop = stop
        
    def __iter__(self):
        i = self.start
        while i<self.stop:
            yield i
            i = i + self.step
    
for i in Range(1, 10, 4):
    print(i)

1
5
9


In [70]:
def f():
    print('A')
    yield 10
    print('B')
    yield 20
    print('C')
    
for elm in f():
    print(elm)

A
10
B
20
C


In [72]:
def my_range(start, stop, step):
    i = start
    while i<stop:
        yield i
        i = i + step
        
for elm in my_range(0, 10, 3):
    print(elm)

0
3
6
9


In [73]:
def tensor_product(l1, l2):
    for elm1 in l1:
        for elm2 in l2:
            yield elm1, elm2

In [74]:
for elm in tensor_product(range(4), range(2)):
    print(elm)

(0, 0)
(0, 1)
(1, 0)
(1, 1)
(2, 0)
(2, 1)
(3, 0)
(3, 1)


In [77]:
class Test:
    def __call__(self):
        print('bonjour')
        
t = Test()
t()

bonjour


In [87]:
class Test:
    a = 1    
    def __init__(self, b=None):
        if b is not None:
            self.b = b

class Test10(Test):
    b = 10
    
class Test20(Test):
    b = 20
    
            
t = Test10(2)

print(t.b)

2


In [95]:
class Biblio:
    def __init__(self, liste_livre=None):
        if liste_livre is None:
            liste_livre = []
        self.liste_livre = liste_livre
        
    
    def append(self, livre):
        self.liste_livre.append(livre)

In [96]:
bib = Biblio()

bib.append('Bonjour')
print(bib.liste_livre)

['Bonjour']


In [97]:
bib2 = Biblio()

bib2.append('Hello')
print(bib2.liste_livre)

['Hello']


2

## Descripteur

Mieux comprendre les méthodes et les property...

<code>
    descr.__get__(self, obj, type=None) -> value
    descr.__set__(self, obj, value) -> None
    descr.__delete__(self, obj) -> None
</code>

Si un atribut de la classe défini la méthode `__get__` alors il devient un descripteur. Cette méthode est appelé à la place de l'attribut. 

In [111]:
#    def set_frequency(self, freq=1000):
#        command = f'FREQ {freq:.3f}'
#        self._conn.write(command)

#    def get_frequency(self):
#        command = f'FREQ?'
#        return float(self._conn.query(command))



class Descripteur:
    def __init__(self, param):
        self.param = param
    
    def __get__(self, obj, type=None):
        command = f'{self.param}?'
        return float(obj._conn.query(command))

    def __set__(self, obj, val):
        command = f'{self.param} {val:.3f}'
        obj._conn.write(command)

    
    
class Instrument():
    def __init__(self, conn):
        self._conn = conn
        
    frequency = Descripteur('FREQ')
    amplitude = Descripteur('AMP')

    
gbf = Instrument(FakeSCPI())
gbf.frequency = 12
print(gbf.frequency)

12.0


In [103]:
type(t).__dict__['a'].__get__(t)

Coucou


In [107]:
def f():
    print('Bonjour')
    
f.__get__

<method-wrapper '__get__' of function object at 0x7fe0336c9e50>

## Créer une class directement / metaclass

* On peut créer une classe avec la fonction type : `type(name, bases, namespace)`

* On peut remplacer cette fonction avec l'argument optionnel metaclass : 

<code>
    class MyClass(metaclass=type):
        pass
</code>

In [29]:
t = Test()

In [32]:
t.get_a()

10

In [31]:
t._a = 10